In [1]:
#pip3 install selenium
#pip3 install pandas
#pip3 install requests
#pip3 install KonNPy : https://velog.io/@heiswicked/M1-Part11-%EB%B3%B5%EB%B6%88%EB%B3%B5%EC%84%A4%EC%B9%98-konlpy.tag-MECAB-on-M1-ver.221230

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Twitter
import pandas as pd
import pyperclip
import requests
import time
import sys, os
import re
import numpy as np
import warnings
import datetime

keys = Keys()
options = Options()
options.add_argument('incognito')
warnings.filterwarnings(action='ignore') 

In [2]:
def open_chrome():
    #네이버 로그인 화면 이동 
    url = "https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com"
    
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    sleep(20)
    
    return driver

def move_site(driver,cafe_url):
    driver = driver 
    driver.get(cafe_url)
    sleep(5)
    
    return driver  

def move_board(driver, number):
    driver = driver 
    css_selector = f"#menuLink{number}"
    driver.find_element(By.CSS_SELECTOR,css_selector).click()
    sleep(5)
    
    return driver

def article_list_n(driver, number):
    driver = driver
    number = number
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.CSS_SELECTOR,"#listSizeSelectDiv > a").click()
    driver.find_element(By.CSS_SELECTOR,f"#listSizeSelectDiv > ul > li:nth-child({number}) > a").click()
    driver.switch_to.default_content()
    sleep(5)
    
    return driver

def get_month_end(month):
    if month == 2:
        return '28'
    elif month in [4, 6, 9, 11]:
        return '30'
    else:
        return '31'
    
def get_month_end_date(year, month):
    end_day = get_month_end(month)
    if month == 2 and ((year % 4 == 0 and year % 100 != 0) or year % 400 == 0):
        end_day = '29'
    return "{0}{1:02d}{2}".format(year, month, end_day)

def get_month_start(year, month):
    return '{:04d}{:02d}01'.format(year, month)

def find_keyword_by_date(driver, keyword, date_start, date_end):
    driver = driver 
    keyword = keyword
    date_start = date_start 
    date_end = date_end 
    
    #날짜 범위 지정
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.CSS_SELECTOR,"#currentSearchDate").click()
    date_s = driver.find_element(By.CSS_SELECTOR,"#input_1")
    date_s.click()
    date_s.send_keys(date_start)
    date_e = driver.find_element(By.CSS_SELECTOR,"#input_2")
    date_e.click()
    date_e.send_keys(date_end)
    
    driver.find_element(By.CSS_SELECTOR,"#btn_set").click()
    
    #키워드 지정
    driver.find_element(By.CSS_SELECTOR,"#currentSearchBy").click()
    driver.find_element(By.CSS_SELECTOR,"#divSearchBy > ul > li:nth-child(2) > a").click()
    
    #키워드 입력
    driver_key = driver.find_element(By.CSS_SELECTOR,"#query")
    driver_key.click()
    driver_key.send_keys(keyword)
    
    #검색
    driver.find_element(By.CSS_SELECTOR,"#main-area > div.list-search > form > div.input_search_area > button").click()
    
    driver.switch_to.default_content()
    
    return driver 
    
def move_page(driver,page):
    driver = driver
    i = page
    
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.XPATH,f"/html/body/div[1]/div/div[7]/a[{i}]").click()
    driver.switch_to.default_content()
    sleep(2)
    
    return driver 

def article_n(driver):
    driver.switch_to.frame("cafe_main")
    article_n = len(driver.find_elements(By.CLASS_NAME,"inner_number"))
    driver.switch_to.default_content()
    
    return article_n

def check_article(driver):
    driver.switch_to.frame("cafe_main")
    n_article = driver.find_elements(By.CLASS_NAME,'nodata')
    driver.switch_to.default_content()
    
    return len(n_article)

def move_article_page(driver, article_n):
    n = article_n
    driver = driver 
    
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.XPATH,f"/html/body/div[1]/div/div[5]/table/tbody/tr[{n}]/td[1]/div[2]/div/a[1]").click()
    driver.switch_to.default_content()
    sleep(2)    
    
    return driver

def reply_extract(driver, cafename, dataframe):
    d = dataframe
    s = cafename
    driver = driver
    
    driver.switch_to.frame("cafe_main")
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    #게시판
    board = soup.select('a.link_board')[0].text.strip()
    
    #게시글
    title = soup.select('h3.title_text')[0].text.strip()
    
    #조회수
    views = soup.select('span.count')[0].text.replace('조회', '')
    
    #작성일
    date = soup.select('span.date')[0].text
    
    #url
    cont_url = driver.find_element(By.XPATH,'//*[@id="spiButton"]').get_attribute('data-url')
    
    driver.switch_to.default_content()
    for i in range(1,len(soup.select('span.text_comment'))+1):
        df = []
        df.append(s)
        df.append(board)
        df.append(title)
        df.append(cont_url)
        df.append(views)
        df.append(date[:10])
        df.append(soup.select('span.text_comment')[i-1].text)
        d = d.append(pd.Series(df,index=d.columns),ignore_index=True)
    return d

def back_page(driver):
    driver = driver 
    
    driver.back()
    sleep(2)
    return driver

In [3]:
cafename = "브랜디드"
cafe_url = "https://cafe.naver.com/coredenim"
board_number = 145
article_by_page = 7
year = 2022
month = 8
keyword = "블루종"

d = pd.DataFrame(columns=["카페명","게시판","게시물","url","조회수","작성날짜","댓글"])
d3 = pd.DataFrame(columns=["카페명","게시판","게시물","url","조회수","작성날짜","댓글"])

date_end = get_month_end_date(year, month)
date_start = get_month_start(year, month)

In [5]:
#driver = open_chrome()
driver = move_site(driver, cafe_url)
driver = move_board(driver, board_number)
driver = article_list_n(driver, article_by_page)

In [ ]:
context = 0
while context > -1 :
    try:
        context+=1
        
    except:
        pass

In [ ]:
context = 7

while context > 0 :
    try:
        context-=1
        if context == 6 :
            
            for i in range(1,11):
                #페이지 이동 
                driver = move_page(driver, i)
                #게시물 확인
                n = article_n(driver)
                if n == 50:
                    for j in range(1,n+1):
                        driver = move_article_page(driver,j)
                        d2 = reply_extract(driver, cafename, d)
                        d3 = d3.append(d2)
                        driver = back_page(driver)
                else:
                    for j in range(1,n+1):
                        driver = move_article_page(driver,j)
                        d2 = reply_extract(driver,cafename,d)
                        d3 = d3.append(d2)
                        driver = back_page(driver)
                        print("완료")
                        
                    context=-1
            
            #다음
            driver = move_page(driver, 11)
            
        else : 
            
            for i in range(2,12):
                #페이지 이동 
                driver = move_page(driver, i)
                
                #게시물 확인
                n = article_n(driver)
                if n == 50:
                    for j in range(1,n+1):
                        driver = move_article_page(driver,j)
                        d2 = reply_extract(driver,cafename,d)
                        d3 = d3.append(d2)
                        driver = back_page(driver)
                        print("완료")
                else:
                    for j in range(1,n+1):
                        driver = move_article_page(driver,j)
                        d2 = reply_extract(driver,cafename,d)
                        d3 = d3.append(d2)
                        driver = back_page(driver)
                        print("완료")
                    context=-1
            #다음
            driver = move_page(driver, 12)
            
    except:
        pass

In [ ]:
#d3 = d3.reset_index()
#d3 = d3.drop(["index"],axis=1)
#d3.to_csv('/Users/jisoo.kim/Downloads/navercafe.csv',encoding='utf-8')

In [14]:
from konlpy.tag import Okt
okt = Okt()
tagger = Mecab()

text = "드이어 환경변수 문제를 해결했다!!"

print(okt.morphs(text, stem=True))

NameError: name 'Mecab' is not defined

In [4]:
d = pd.read_csv('/Users/jisoo.kim/Downloads/navercafe.csv',encoding='utf-8')

In [13]:
d.iloc[2][7]

'비긴202 트위드 이뻐용'

In [12]:
d

,Unnamed: 0,카페명,게시판,게시물,url,조회수,작성날짜,댓글
0,0,브랜디드,패션,자신의 최애 블루종 착샷을 보여주세요 ㅎㅎ,https://cafe.naver.com/coredenim/1131472,"1,209",2022.08.31,이 블루종 너무이쁜걸요 ㅎㅎ
1,1,브랜디드,패션,자신의 최애 블루종 착샷을 보여주세요 ㅎㅎ,https://cafe.naver.com/coredenim/1131472,"1,209",2022.08.31,포커페이스!
2,2,브랜디드,패션,자신의 최애 블루종 착샷을 보여주세요 ㅎㅎ,https://cafe.naver.com/coredenim/1131472,"1,209",2022.08.31,비긴202 트위드 이뻐용
3,3,브랜디드,패션,자신의 최애 블루종 착샷을 보여주세요 ㅎㅎ,https://cafe.naver.com/coredenim/1131472,"1,209",2022.08.31,사진고자라 아워스코프임
4,4,브랜디드,패션,자신의 최애 블루종 착샷을 보여주세요 ㅎㅎ,https://cafe.naver.com/coredenim/1131472,"1,209",2022.08.31,쿠어 울 블루종 !
...,...,...,...,...,...,...,...,...
981,981,브랜디드,패션,노운 1치킨 블루종 들어가신분 없으실까요..?,https://cafe.naver.com/coredenim/1106171,820,2022.08.01,1사 밖에없어서 포기 ㅠ ㅠ
982,982,브랜디드,패션,노운 1치킨 블루종 들어가신분 없으실까요..?,https://cafe.naver.com/coredenim/1106171,820,2022.08.01,2사이즈ㅠㅠㅠ
983,983,브랜디드,패션,노운 1치킨 블루종 들어가신분 없으실까요..?,https://cafe.naver.com/coredenim/1106171,820,2022.08.01,1사 구매 했는데 바로 품절이네요..?
984,984,브랜디드,패션,노운 1치킨 블루종 들어가신분 없으실까요..?,https://cafe.naver.com/coredenim/1106171,820,2022.08.01,와..오늘 두시까지만해도 원가였는데 갑자기 3만원..? 이걸 놓쳤네


In [18]:
from konlpy.tag import Mecab

In [19]:
tokenizer = Mecab()

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [17]:

!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

/dev/fd/12: line 48: python: command not found
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   450k      0  0:00:03  0:00:03 --:--:--  707k
x mecab-0.996-ko-0.9.2/
x mecab-0.996-ko-0.9.2/example/
x mecab-0.996-ko-0.9.2/example/example.cpp
x mecab-0.996-ko-0.9.2/example/example_lattice.cpp
x mecab-0.996-ko-0.9.2/example/example_lattice.c
x mecab-0.996-ko-0.9.2/example/example.c
x mecab-0.996-ko-0.9.2/example/thread_test.cpp
x mecab-0.996-ko-0.9.2/mecab-config.in
x mecab-0.996-ko-0.9.2/man/
x mecab-0.996-ko-0.9.2/man/Makefile.am
x mecab-0.996-ko-0.9.2/man/mecab.1
x mecab-0.996-ko-0.9.2/man/Makefile.in
x mecab-0.996-ko-0.9.2/mecab.iss.in
x mecab-0.996-ko-0.9.2/config.guess
x mecab-0.996-ko-0.9.2/README
x mecab-0.996-ko-0.9.2/COPYING
x mecab-0.996-ko-0.9

x mecab-0.996-ko-0.9.2/doc/doxygen/open.png
x mecab-0.996-ko-0.9.2/doc/doxygen/mecab_8h.html
x mecab-0.996-ko-0.9.2/doc/dic.html
x mecab-0.996-ko-0.9.2/doc/partial.html
x mecab-0.996-ko-0.9.2/doc/feature.png
x mecab-0.996-ko-0.9.2/doc/Makefile.in
x mecab-0.996-ko-0.9.2/missing
x mecab-0.996-ko-0.9.2/BSD
x mecab-0.996-ko-0.9.2/NEWS
x mecab-0.996-ko-0.9.2/mkinstalldirs
x mecab-0.996-ko-0.9.2/src/
x mecab-0.996-ko-0.9.2/src/dictionary.h
x mecab-0.996-ko-0.9.2/src/writer.h
x mecab-0.996-ko-0.9.2/src/utils.h
x mecab-0.996-ko-0.9.2/src/string_buffer.cpp
x mecab-0.996-ko-0.9.2/src/tokenizer.cpp
x mecab-0.996-ko-0.9.2/src/make.bat
x mecab-0.996-ko-0.9.2/src/mecab.h
x mecab-0.996-ko-0.9.2/src/freelist.h
x mecab-0.996-ko-0.9.2/src/string_buffer.h
x mecab-0.996-ko-0.9.2/src/learner_tagger.h
x mecab-0.996-ko-0.9.2/src/dictionary_compiler.cpp
x mecab-0.996-ko-0.9.2/src/eval.cpp
x mecab-0.996-ko-0.9.2/src/mecab-system-eval.cpp
x mecab-0.996-ko-0.9.2/src/darts.h
x mecab-0.996-ko-0.9.2/src/param.h
x m

checking for shared library run path origin... done
checking for iconv... yes
checking for working iconv... yes
checking how to link with libiconv... -liconv
checking for iconv declaration... 
         extern size_t iconv (iconv_t cd, char * *inbuf, size_t *inbytesleft, char * *outbuf, size_t *outbytesleft);
checking for ANSI C header files... (cached) yes
checking for an ANSI C-conforming const... yes
checking whether byte ordering is bigendian... (cached) no
checking for string.h... (cached) yes
checking for stdlib.h... (cached) yes
checking for unistd.h... (cached) yes
checking fcntl.h usability... yes
checking fcntl.h presence... yes
checking for fcntl.h... yes
checking for stdint.h... (cached) yes
checking for sys/stat.h... (cached) yes
checking sys/mman.h usability... yes
checking sys/mman.h presence... yes
checking for sys/mman.h... yes
checking sys/times.h usability... yes
checking sys/times.h presence... yes
checking for sys/times.h... yes
checking for sys/types.h... (cached) 

In file included from viterbi.cpp:14:
./param.h:30:13: warning: unused function 'lexical_cast<std::string, std::string>' [-Wunused-function]
std::string lexical_cast<std::string, std::string>(std::string arg) {
            ^
8 warnings generated.
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c viterbi.cpp -o viterbi.o >/dev/null 2>&1
/bin/sh ../libtool --tag=CXX   --mode=compile g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102   -DMECAB_DEFAULT_RC="\"/usr/local/etc/mecabrc\""    -O3 -Wall  -c -o tagger.lo tagger.cpp
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c tagger.cpp  -fno-common -DPIC -o .libs/tagger.o
In file included from tagger.cpp:10:
In file included from ./connector.h:10:
In file included from ./mmap.h:51:
./utils.h:47:8: warning: 'sprintf' is deprecated: This function is provided for compatibility reasons only.  Due to se

1 warning generated.
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c utils.cpp -o utils.o >/dev/null 2>&1
/bin/sh ../libtool --tag=CXX   --mode=compile g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102   -DMECAB_DEFAULT_RC="\"/usr/local/etc/mecabrc\""    -O3 -Wall  -c -o eval.lo eval.cpp
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c eval.cpp  -fno-common -DPIC -o .libs/eval.o
In file included from eval.cpp:14:
In file included from ./stream_wrapper.h:12:
./utils.h:47:8: warning: 'sprintf' is deprecated: This function is provided for compatibility reasons only.  Due to security concerns inherent in the design of sprintf(3), it is highly recommended that you use snprintf(3) instead. [-Wdeprecated-declarations]
  std::sprintf(s, "%-16f", val);
       ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/stdio.h:188:1: note: 's

...

2 warnings generated.
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c writer.cpp -o writer.o >/dev/null 2>&1
/bin/sh ../libtool --tag=CXX   --mode=compile g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102   -DMECAB_DEFAULT_RC="\"/usr/local/etc/mecabrc\""    -O3 -Wall  -c -o string_buffer.lo string_buffer.cpp
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c string_buffer.cpp  -fno-common -DPIC -o .libs/string_buffer.o
In file included from string_buffer.cpp:9:
In file included from ./string_buffer.h:11:
./utils.h:47:8: warning: 'sprintf' is deprecated: This function is provided for compatibility reasons only.  Due to security concerns inherent in the design of sprintf(3), it is highly recommended that you use snprintf(3) instead. [-Wdeprecated-declarations]
  std::sprintf(s, "%-16f", val);
       ^
/Library/Developer/CommandLineTools/SDKs/

7 warnings generated.
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c tokenizer.cpp -o tokenizer.o >/dev/null 2>&1
/bin/sh ../libtool --tag=CXX   --mode=compile g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102   -DMECAB_DEFAULT_RC="\"/usr/local/etc/mecabrc\""    -O3 -Wall  -c -o char_property.lo char_property.cpp
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c char_property.cpp  -fno-common -DPIC -o .libs/char_property.o
In file included from char_property.cpp:11:
In file included from ./char_property.h:8:
In file included from ./mmap.h:51:
./utils.h:47:8: warning: 'sprintf' is deprecated: This function is provided for compatibility reasons only.  Due to security concerns inherent in the design of sprintf(3), it is highly recommended that you use snprintf(3) instead. [-Wdeprecated-declarations]
  std::sprintf(s, "%-16f", val);
       ^


/bin/sh ../libtool --tag=CXX   --mode=compile g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102   -DMECAB_DEFAULT_RC="\"/usr/local/etc/mecabrc\""    -O3 -Wall  -c -o learner.lo learner.cpp
libtool: compile:  g++ -DHAVE_CONFIG_H -I. -I.. -DDIC_VERSION=102 -DMECAB_DEFAULT_RC=\"/usr/local/etc/mecabrc\" -O3 -Wall -c learner.cpp  -fno-common -DPIC -o .libs/learner.o
In file included from learner.cpp:10:
In file included from ./feature_index.h:12:
In file included from ./mmap.h:51:
./utils.h:47:8: warning: 'sprintf' is deprecated: This function is provided for compatibility reasons only.  Due to security concerns inherent in the design of sprintf(3), it is highly recommended that you use snprintf(3) instead. [-Wdeprecated-declarations]
  std::sprintf(s, "%-16f", val);
       ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/stdio.h:188:1: note: 'sprintf' has been explicitly marked deprecated here
__deprecated_msg("This function is provided for compatibility reasons only.  Due to

/bin/sh ../libtool --tag=CXX   --mode=link g++  -O3 -Wall  -no-undefined -version-info 2:0:0  -o libmecab.la -rpath /usr/local/lib viterbi.lo tagger.lo utils.lo eval.lo iconv_utils.lo dictionary_rewriter.lo dictionary_generator.lo dictionary_compiler.lo context_id.lo connector.lo nbest_generator.lo writer.lo string_buffer.lo param.lo tokenizer.lo char_property.lo dictionary.lo feature_index.lo lbfgs.lo learner_tagger.lo learner.lo libmecab.lo  -lpthread -lpthread  -lstdc++ -liconv
libtool: link: g++ -dynamiclib  -o .libs/libmecab.2.dylib  .libs/viterbi.o .libs/tagger.o .libs/utils.o .libs/eval.o .libs/iconv_utils.o .libs/dictionary_rewriter.o .libs/dictionary_generator.o .libs/dictionary_compiler.o .libs/context_id.o .libs/connector.o .libs/nbest_generator.o .libs/writer.o .libs/string_buffer.o .libs/param.o .libs/tokenizer.o .libs/char_property.o .libs/dictionary.o .libs/feature_index.o .libs/lbfgs.o .libs/learner_tagger.o .libs/learner.o .libs/libmecab.o   -lpthread -lstdc++ -liconv 

iter=3 err=0.85294 F=0.89213 target=596.72480 diff=0.30881
iter=4 err=0.61765 F=0.95467 target=336.30744 diff=0.43641
iter=5 err=0.50000 F=0.96702 target=246.53039 diff=0.26695
iter=6 err=0.35294 F=0.95472 target=188.93963 diff=0.23361
iter=7 err=0.20588 F=0.99106 target=168.62665 diff=0.10751
iter=8 err=0.05882 F=0.99777 target=158.64865 diff=0.05917
iter=9 err=0.08824 F=0.99665 target=154.14530 diff=0.02839
iter=10 err=0.08824 F=0.99665 target=151.94257 diff=0.01429
iter=11 err=0.02941 F=0.99888 target=147.20825 diff=0.03116
iter=12 err=0.00000 F=1.00000 target=147.34956 diff=0.00096
iter=13 err=0.02941 F=0.99888 target=146.32592 diff=0.00695
iter=14 err=0.00000 F=1.00000 target=145.77299 diff=0.00378
iter=15 err=0.02941 F=0.99888 target=145.24641 diff=0.00361
iter=16 err=0.00000 F=1.00000 target=144.96490 diff=0.00194
iter=17 err=0.02941 F=0.99888 target=144.90246 diff=0.00043
iter=18 err=0.00000 F=1.00000 target=144.75959 diff=0.00099
iter=19 err=0.00000 F=1.00000 target=144.71727 